In [ ]:
import pandas as pd
import pathlib
import networkx
import matplotlib.pyplot as plt

from drawing.silhouettes import calculate_and_plot_silhouettes
from networks.TemporalNetwork import TemporalNetwork

## Declare parameters to use throughout

In [ ]:
distance_metric = 'jaccard'
cluster_limit_type = 'maxclust'
cluster_limit_range = [2 + (1 * i) for i in range(0, 10)]

cluster_methods = ['k_means', 'ward', 'median', 'centroid', 'weighted', 'average', 'complete', 'single']

events = [(12 * i, '', 'dashed') for i in range(4 + 1)]
phases = [
    (0, 12, 'Day 1'),
    (12, 24, 'Night 1'),
    (24, 36, 'Day 2'),
    (36, 48, 'Night 2')
]

output_directory = None

### Load/create temporal network

In [ ]:
node_table_filepath = '../data/temporal_data/circadian/circadian_temporal_node_data_mean_48.csv'
static_network_filepath = '../data/static_networks/circadian_net.edgelist'
binary = True
normalise = True
threshold = 0.5

node_table = pd.read_csv(node_table_filepath, sep='\t', index_col=0)
static_network = networkx.read_edgelist(static_network_filepath)

temporal_network = TemporalNetwork.from_static_network_and_node_table_dataframe(
    static_network,
    node_table,
    combine_node_weights=lambda x, y: x*y,
    threshold=threshold,
    binary=binary,
    normalise=normalise)

### Plot silhouettes across different cluster methods

In [ ]:
gridspec_kw = {"width_ratios": [3, 1, 2]}
figsize = (9, 4*len(cluster_methods))
fig, axs = plt.subplots(len(cluster_methods), 3, figsize=figsize, gridspec_kw=gridspec_kw, sharey=True)

i = 0
for cluster_method in cluster_methods:
    row = (axs[i, 0], axs[i, 1], axs[i, 2])
    time_ticks = [6 * i for i in range(8 + 1)]
    plotted = calculate_and_plot_silhouettes(
        row, temporal_network, cluster_method, distance_metric, cluster_limit_type, cluster_limit_range, events,
        phases, variable_name='cluster_method', variable=cluster_method, time_ticks=time_ticks)
    if plotted:
        i += 1

for j in range(i, len(cluster_methods)):
    for ax in (axs[j, 0], axs[j, 1], axs[j, 2]):
        fig.delaxes(ax)

title = f'data={pathlib.Path(node_table_filepath).stem}, binary={binary}, normalise={normalise}, threshold={threshold}'
title += f'\nmetric={distance_metric}, cluster_limit_type={cluster_limit_type} '
fig.suptitle(title, y=0.91, weight='bold')

# Save
if output_directory is not None:
    filename = f"{output_directory}/silhouette_scores_over_cluster_methods"
    plt.savefig(f"{filename}.png", dpi=250, bbox_inches="tight")
